# 事件策略研究工具使用案例
---


环境配置说明：

* python == 3.12


## 一、CZSC对象的使用

CZSC对象为缠论形态、力度分析提供了最底层的支持。

In [1]:
import czsc
import pandas as pd

C:\Users\zengb\.czsc 目录缓存超过1GB，请适当清理。调用 czsc.empty_cache_path() 可以直接清空缓存


In [2]:
# mock 生成15分钟K线数据
dfk = czsc.mock.generate_symbol_kines(symbol="test", freq="15分钟", sdt="20220101", edt="20250101")
dfk = dfk[['dt', 'symbol', 'open', 'high', 'low', 'close', 'vol', 'amount']].copy().reset_index(drop=True)

# 将标准K线数据转成 RawBar 对象列表
raw_bars = czsc.format_standard_kline(dfk, freq='15分钟')

# 初始化 CZSC 对象
czsc_obj = czsc.CZSC(raw_bars, max_bi_num=30)

In [3]:
# 查看 CZSC 对象的基本信息
print(czsc_obj)

print("\n最近5笔走势：")
for bi in czsc_obj.bi_list[-5:]:
    print(bi)
    
print("\n笔的常用属性：")
last_bi = czsc_obj.bi_list[-1]
print(f"笔的方向: {last_bi.direction}； 开始时间: {last_bi.sdt}； 结束时间: {last_bi.edt}")
print(f"笔的斜率: {last_bi.slope}； SNR: {last_bi.SNR}； 笔的强度: {last_bi.power}")

CZSC(symbol=test, freq=F15, max_bi_num=30, bi_count=30)

最近5笔走势：
BI(symbol=test, sdt=2024-12-27 13:15:00, edt=2024-12-29 11:30:00, direction=Down, high=177.12, low=168.89)
BI(symbol=test, sdt=2024-12-29 11:30:00, edt=2024-12-30 13:30:00, direction=Up, high=176.86, low=168.89)
BI(symbol=test, sdt=2024-12-30 13:30:00, edt=2024-12-30 15:00:00, direction=Down, high=176.86, low=172.94)
BI(symbol=test, sdt=2024-12-30 15:00:00, edt=2024-12-31 14:45:00, direction=Up, high=183.52, low=172.94)
BI(symbol=test, sdt=2024-12-31 14:45:00, edt=2025-01-01 11:00:00, direction=Down, high=183.52, low=180.04)

笔的常用属性：
笔的方向: 向下； 开始时间: 2024-12-31 14:45:00； 结束时间: 2025-01-01 11:00:00
笔的斜率: 0.05843137254902003； SNR: 0.13138686131386865； 笔的强度: 3.48


In [7]:
# czsc_obj 可视化
def plot_czsc_chart(czsc_obj, **kwargs):
    """使用 plotly 绘制K线分析图"""
    import pandas as pd
    from czsc.utils.plotly_plot import KlineChart
    
    height = kwargs.get('height', 600)
    ma_system = kwargs.get('ma_system', (5, 10, 21, 34, 55, 89, 144))

    bi_list = czsc_obj.bi_list
    df = pd.DataFrame([x.__dict__ for x in czsc_obj.bars_raw])
    df = df[['dt', 'symbol', 'open', 'high', 'low', 'close', 'vol', 'amount']]
    chart = KlineChart(n_rows=3, title="{}-{}".format(czsc_obj.symbol, czsc_obj.freq.value), height=height)
    chart.add_kline(df, name="")
    chart.add_sma(df, ma_seq=[ma_system[0]], row=1, visible=True, line_width=1.2)
    chart.add_sma(df, ma_seq=ma_system[1:], row=1, visible=False, line_width=1.2)
    chart.add_vol(df, row=2)
    chart.add_macd(df, row=3)

    if len(bi_list) > 0:
        bi1 = [{'dt': x.fx_a.dt, "bi": x.fx_a.fx, "text": x.fx_a.mark.value.replace("分型", "")} for x in bi_list]
        bi2 = [{'dt': bi_list[-1].fx_b.dt, "bi": bi_list[-1].fx_b.fx, "text": bi_list[-1].fx_b.mark.value[0]}]
        bi = pd.DataFrame(bi1 + bi2)
        fx = pd.DataFrame([{'dt': x.dt, "fx": x.fx} for x in czsc_obj.fx_list])
        
        # 分型用虚线表示
        chart.add_scatter_indicator(fx['dt'], fx['fx'], name="分型", row=1, line_width=1.8, line_dash='dash')
        chart.add_scatter_indicator(bi['dt'], bi['bi'], name="笔", text=bi['text'], row=1, line_width=1.8)
    return chart

chart = plot_czsc_chart(czsc_obj)
chart.fig.show()

## 二、事件策略构建